In [53]:
# Importing libraries

# These are the normal libraries
import pandas as pd
import numpy as np

# This is just so that we don't get annoying warnings
import warnings
warnings.filterwarnings('ignore')

# This is the most common viz library in python
import matplotlib.pyplot as plt
%matplotlib inline

# This one is the above on steroids
import seaborn as sns

from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

# These Libs are for stats
import statsmodels.api as sm
from statsmodels.formula.api import ols


In [54]:
# Clean and fill the NA value and order per month

url = 'https://docs.google.com/spreadsheets/d/1a9g5MfgAJ17HiYYKsjBi2yaO_HMGLIAX/edit?usp=sharing&ouid=108054823958769193495&rtpof=true&sd=true'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
mark_cust_A = pd.read_excel(path)

# fonction processing all the steps to make the data clean for our file
def Data_quarter_Y(mark_cust_A):
    # creation of a list to store columns name after lower and removing spaces
    cols_st = []
    # 
    for i in range(len(mark_cust_A.columns)):
        cols_st.append(mark_cust_A.columns[i].lower().replace(" ","_"))
    mark_cust_A.columns = cols_st
    mark_cust_A = mark_cust_A.rename(columns = {"employmentstatus":"employment_status"})
    mark_cust_A = mark_cust_A.drop({"unnamed:_0","state" },axis=1)
    mark_cust_A = mark_cust_A.fillna({'response':'No','number_of_open_complaints': 1,'vehicle_type':'Other vehicule','months_since_last_claim':mark_cust_A['months_since_last_claim'].mean(),'vehicle_size':'Medsize'})
    mark_cust_A['date'] = pd.to_datetime(mark_cust_A['effective_to_date'])
    mark_cust_A['month'] = mark_cust_A['date'].dt.to_period('M')
    return mark_cust_A
    
mark_cust_A = Data_quarter_Y(mark_cust_A)

mark_cust_A


,customer,customer_lifetime_value,response,coverage,education,effective_to_date,employment_status,gender,income,location_code,...,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type,date,month
0,DK49336,4809.216960,No,Basic,College,2011-02-18,Employed,M,48029,Suburban,...,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,Other vehicule,2011-02-18,2011-02
1,KX64629,2228.525238,No,Basic,College,2011-01-18,Unemployed,F,0,Suburban,...,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,Other vehicule,2011-01-18,2011-01
2,LZ68649,14947.917300,No,Basic,Bachelor,2011-02-10,Employed,M,22139,Suburban,...,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A,2011-02-10,2011-02
3,XL78013,22332.439460,Yes,Extended,College,2011-01-11,Employed,M,49078,Suburban,...,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A,2011-01-11,2011-01
4,QA50777,9025.067525,No,Premium,Bachelor,2011-01-17,Medical Leave,F,23675,Suburban,...,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,Other vehicule,2011-01-17,2011-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,FE99816,15563.369440,No,Premium,Bachelor,2011-01-19,Unemployed,F,0,Suburban,...,Personal Auto,Personal L1,Offer3,Web,1214.400000,Luxury Car,Medsize,A,2011-01-19,2011-01
10906,KX53892,5259.444853,No,Basic,College,2011-01-06,Employed,F,61146,Urban,...,Personal Auto,Personal L3,Offer2,Branch,273.018929,Four-Door Car,Medsize,A,2011-01-06,2011-01
10907,TL39050,23893.304100,No,Extended,Bachelor,2011-02-06,Employed,F,39837,Rural,...,Corporate Auto,Corporate L3,Offer1,Web,381.306996,Luxury SUV,Medsize,Other vehicule,2011-02-06,2011-02
10908,WA60547,11971.977650,No,Premium,College,2011-02-13,Employed,F,64195,Urban,...,Personal Auto,Personal L1,Offer1,Branch,618.288849,SUV,Medsize,A,2011-02-13,2011-02


In [55]:
mark_cust_A.isna().sum()
mark_cust_A['vehicle_class'].unique()
mark_cust_A['vehicle_class'] = mark_cust_A['vehicle_class'].fillna('Car')
mark_cust_A

,customer,customer_lifetime_value,response,coverage,education,effective_to_date,employment_status,gender,income,location_code,...,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type,date,month
0,DK49336,4809.216960,No,Basic,College,2011-02-18,Employed,M,48029,Suburban,...,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,Other vehicule,2011-02-18,2011-02
1,KX64629,2228.525238,No,Basic,College,2011-01-18,Unemployed,F,0,Suburban,...,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,Other vehicule,2011-01-18,2011-01
2,LZ68649,14947.917300,No,Basic,Bachelor,2011-02-10,Employed,M,22139,Suburban,...,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A,2011-02-10,2011-02
3,XL78013,22332.439460,Yes,Extended,College,2011-01-11,Employed,M,49078,Suburban,...,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A,2011-01-11,2011-01
4,QA50777,9025.067525,No,Premium,Bachelor,2011-01-17,Medical Leave,F,23675,Suburban,...,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,Other vehicule,2011-01-17,2011-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,FE99816,15563.369440,No,Premium,Bachelor,2011-01-19,Unemployed,F,0,Suburban,...,Personal Auto,Personal L1,Offer3,Web,1214.400000,Luxury Car,Medsize,A,2011-01-19,2011-01
10906,KX53892,5259.444853,No,Basic,College,2011-01-06,Employed,F,61146,Urban,...,Personal Auto,Personal L3,Offer2,Branch,273.018929,Four-Door Car,Medsize,A,2011-01-06,2011-01
10907,TL39050,23893.304100,No,Extended,Bachelor,2011-02-06,Employed,F,39837,Rural,...,Corporate Auto,Corporate L3,Offer1,Web,381.306996,Luxury SUV,Medsize,Other vehicule,2011-02-06,2011-02
10908,WA60547,11971.977650,No,Premium,College,2011-02-13,Employed,F,64195,Urban,...,Personal Auto,Personal L1,Offer1,Branch,618.288849,SUV,Medsize,A,2011-02-13,2011-02


In [56]:
Numerical = mark_cust_A.select_dtypes(include='number').reset_index()


Categorical = mark_cust_A.select_dtypes(include=object).reset_index()

Numerical

,index,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
0,0,4809.216960,48029,61,7.000000,52,0.0,9,292.800000
1,1,2228.525238,0,64,3.000000,26,0.0,1,744.924331
2,2,14947.917300,22139,100,34.000000,31,0.0,2,480.000000
3,3,22332.439460,49078,97,10.000000,3,0.0,2,484.013411
4,4,9025.067525,23675,117,15.149071,31,1.0,7,707.925645
...,...,...,...,...,...,...,...,...,...
10905,10905,15563.369440,0,253,15.149071,40,1.0,7,1214.400000
10906,10906,5259.444853,61146,65,7.000000,68,0.0,6,273.018929
10907,10907,23893.304100,39837,201,11.000000,63,0.0,2,381.306996
10908,10908,11971.977650,64195,158,0.000000,27,4.0,6,618.288849


In [57]:
# encoding categorical data

Categorical = Categorical.drop(['customer'], axis = 1)
# chane value for object to 0 and 1
New_Categorical = pd.get_dummies(Categorical)


New_df = pd.concat([New_Categorical, Numerical], axis = 1).reset_index()

New_df = New_df.drop(['level_0','index'], axis = 1)

New_df

,response_No,response_Yes,coverage_Basic,coverage_Extended,coverage_Premium,education_Bachelor,education_College,education_Doctor,education_High School or Below,education_Master,...,vehicle_type_A,vehicle_type_Other vehicule,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
0,1,0,1,0,0,0,1,0,0,0,...,0,1,4809.216960,48029,61,7.000000,52,0.0,9,292.800000
1,1,0,1,0,0,0,1,0,0,0,...,0,1,2228.525238,0,64,3.000000,26,0.0,1,744.924331
2,1,0,1,0,0,1,0,0,0,0,...,1,0,14947.917300,22139,100,34.000000,31,0.0,2,480.000000
3,0,1,0,1,0,0,1,0,0,0,...,1,0,22332.439460,49078,97,10.000000,3,0.0,2,484.013411
4,1,0,0,0,1,1,0,0,0,0,...,0,1,9025.067525,23675,117,15.149071,31,1.0,7,707.925645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,1,0,0,0,1,1,0,0,0,0,...,1,0,15563.369440,0,253,15.149071,40,1.0,7,1214.400000
10906,1,0,1,0,0,0,1,0,0,0,...,1,0,5259.444853,61146,65,7.000000,68,0.0,6,273.018929
10907,1,0,0,1,0,1,0,0,0,0,...,0,1,23893.304100,39837,201,11.000000,63,0.0,2,381.306996
10908,1,0,0,0,1,0,1,0,0,0,...,1,0,11971.977650,64195,158,0.000000,27,4.0,6,618.288849


In [58]:
Numerical = Numerical.drop(['index'], axis = 1)

In [59]:
# I will round numerical numbers to get better result 

Numerical
Numerical['customer_lifetime_value'] = Numerical['customer_lifetime_value'].round(1)

Numerical['total_claim_amount'] = Numerical['total_claim_amount'].round(1)

Numerical['months_since_last_claim'] = Numerical['months_since_last_claim'].round(1)



New_df = pd.concat([New_Categorical, Numerical], axis = 1)



New_df.drop(['index'], axis = 1)






,response_No,response_Yes,coverage_Basic,coverage_Extended,coverage_Premium,education_Bachelor,education_College,education_Doctor,education_High School or Below,education_Master,...,vehicle_type_A,vehicle_type_Other vehicule,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
0,1,0,1,0,0,0,1,0,0,0,...,0,1,4809.2,48029,61,7.0,52,0.0,9,292.8
1,1,0,1,0,0,0,1,0,0,0,...,0,1,2228.5,0,64,3.0,26,0.0,1,744.9
2,1,0,1,0,0,1,0,0,0,0,...,1,0,14947.9,22139,100,34.0,31,0.0,2,480.0
3,0,1,0,1,0,0,1,0,0,0,...,1,0,22332.4,49078,97,10.0,3,0.0,2,484.0
4,1,0,0,0,1,1,0,0,0,0,...,0,1,9025.1,23675,117,15.1,31,1.0,7,707.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,1,0,0,0,1,1,0,0,0,0,...,1,0,15563.4,0,253,15.1,40,1.0,7,1214.4
10906,1,0,1,0,0,0,1,0,0,0,...,1,0,5259.4,61146,65,7.0,68,0.0,6,273.0
10907,1,0,0,1,0,1,0,0,0,0,...,0,1,23893.3,39837,201,11.0,63,0.0,2,381.3
10908,1,0,0,0,1,0,1,0,0,0,...,1,0,11972.0,64195,158,0.0,27,4.0,6,618.3


In [60]:
# Normalisation of the data

from sklearn.preprocessing import MinMaxScaler

num_data = New_df.drop(['total_claim_amount'], axis=1)
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(num_data)
normalized_data = pd.DataFrame(normalized_data, columns=num_data.columns)
normalized_data

,index,response_No,response_Yes,coverage_Basic,coverage_Extended,coverage_Premium,education_Bachelor,education_College,education_Doctor,education_High School or Below,...,vehicle_size_Small,vehicle_type_A,vehicle_type_Other vehicule,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies
0,0.000000,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.035752,0.480381,0.000000,0.200000,0.525253,0.0,1.000
1,0.000092,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.004059,0.000000,0.012658,0.085714,0.262626,0.0,0.000
2,0.000183,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.160264,0.221432,0.164557,0.971429,0.313131,0.0,0.125
3,0.000275,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.250952,0.490873,0.151899,0.285714,0.030303,0.0,0.125
4,0.000367,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.087527,0.236795,0.236287,0.431429,0.313131,0.2,0.750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,0.999633,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.167823,0.000000,0.810127,0.431429,0.404040,0.2,0.750
10906,0.999725,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.041281,0.611576,0.016878,0.200000,0.686869,0.0,0.625
10907,0.999817,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.270122,0.398446,0.590717,0.314286,0.636364,0.0,0.125
10908,0.999908,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.123718,0.642072,0.409283,0.000000,0.272727,0.8,0.625


In [76]:


# Extract the 'total_claim_amount' column from New_df
total_claim_amount_column = New_df['total_claim_amount']

# Concatenate the 'total_claim_amount' column with normalized_data
New_df = pd.concat([normalized_data, total_claim_amount_column], axis=1)

# The 'total_claim_amount' column is concatenated to the normalized_data

New_df = New_df.drop(['index'], axis = 1)


In [79]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures


# Split the data into features (X) and the target variable (Y)
Y = New_df['total_claim_amount']
X = New_df.drop(['total_claim_amount'], axis=1)

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [82]:
# apply linear regression on the following data

# The linear model has the formula Y = a * X + b, with 'add_constant' we force the model
# to have an intercept because by default on statsmodel library the `linear models`
# doesn't have intercept!!!
X = sm.add_constant(X)
model = sm.OLS(Y,X).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     total_claim_amount   R-squared:                       0.771
Model:                            OLS   Adj. R-squared:                  0.770
Method:                 Least Squares   F-statistic:                     794.3
Date:                Tue, 07 Nov 2023   Prob (F-statistic):               0.00
Time:                        12:50:22   Log-Likelihood:                -69384.
No. Observations:               10910   AIC:                         1.389e+05
Df Residuals:                   10863   BIC:                         1.392e+05
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const                           -1.101e+14   1.53e+14     -0.721      0.471    -4.1e+14     1.9e+14
response_No                     -5.681e+13   7.88e+13     -0.721      0.471   -2.11e+14    9.77e+13
response_Yes                    -5.681e+13   7.88e+13     -0.721      0.471   -2.11e+14    9.77e+13
coverage_Basic                  -4.064e+13   5.64e+13     -0.721      0.471   -1.51e+14    6.99e+13
coverage_Extended               -4.064e+13   5.64e+13     -0.721      0.471   -1.51e+14    6.99e+13
coverage_Premium                -4.064e+13   5.64e+13     -0.721      0.471   -1.51e+14    6.99e+13
education_Bachelor              -1.065e+13   1.48e+13     -0.721      0.471   -3.96e+13    1.83e+13
education_College               -1.065e+13   1.48e+13     -0.721      0.471   -3.96e+13    1.83e+13
education_Doctor                -1.065e+13   1.48e+13     -0.721      0.471   -3.96e+13    1.83e+13
education_High School or Below  -1.065e+13   1.48e+13     -0.721      0.471   -3.96e+13    1.83e+13
education_Master                -1.065e+13   1.48e+13     -0.721      0.471   -3.96e+13    1.83e+13
employment_status_Disabled       1.222e+13    1.7e+13      0.721      0.471    -2.1e+13    4.55e+13
employment_status_Employed       1.222e+13    1.7e+13      0.721      0.471    -2.1e+13    4.55e+13
employment_status_Medical Leave  1.222e+13    1.7e+13      0.721      0.471    -2.1e+13    4.55e+13
employment_status_Retired        1.222e+13    1.7e+13      0.721      0.471    -2.1e+13    4.55e+13
employment_status_Unemployed     1.222e+13    1.7e+13      0.721      0.471    -2.1e+13    4.55e+13
gender_F                         3.388e+13    4.7e+13      0.721      0.471   -5.83e+13    1.26e+14
gender_M                         3.388e+13    4.7e+13      0.721      0.471   -5.83e+13    1.26e+14
location_code_Rural              2.151e+13   2.99e+13      0.721      0.471    -3.7e+13       8e+13
location_code_Suburban           2.151e+13   2.99e+13      0.721      0.471    -3.7e+13       8e+13
location_code_Urban              2.151e+13   2.99e+13      0.721      0.471    -3.7e+13       8e+13
marital_status_Divorced          2.101e+13   2.92e+13      0.721      0.471   -3.61e+13    7.82e+13
marital_status_Married           2.101e+13   2.92e+13      0.721      0.471   -3.61e+13    7.82e+13
marital_status_Single            2.101e+13   2.92e+13      0.721      0.471   -3.61e+13    7.82e+13
policy_type_Corporate Auto       2.417e+13   3.35e+13      0.721      0.471   -4.16e+13    8.99e+13
policy_type_Personal Auto        2.171e+13   3.01e+13      0.721      0.471   -3.74e+13    8.08e+13
policy_type_Special Auto         2.238e+13   3.11e+13      0.721      0.471   -3.85e+13    8.33e+13
policy_Corporate L1              5.231e+12   7.26e+12      0.721      0.471      -9

In [97]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
 
#  Apply polynomial features to capture non-linear relationships
poly = PolynomialFeatures(degree=2) 
X_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

 # Create and train a linear regression model
model = LinearRegression()
model.fit(X_poly, Y_train)

Y_pred = model.predict(X_test_poly)




In [98]:
# Evaluate the model
r2 = r2_score(Y_test, Y_pred)
mse = mean_squared_error(Y_test, Y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y_test, Y_pred)

# Print evaluation metrics
print("R-squared (R2) =", round(r2, 4))
print("Mean Squared Error (MSE) =", round(mse, 2))
print("Root Mean Squared Error (RMSE) =", round(rmse, 2))
print("Mean Absolute Error (MAE) =", round(mae, 2))

R-squared (R2) = -2.4086802244912767e+19
Mean Squared Error (MSE) = 1.9448493480996368e+24
Root Mean Squared Error (RMSE) = 1394578555729.16
Mean Absolute Error (MAE) = 43846493274.19


In [107]:
X_train.columns


Index(['response_No', 'response_Yes', 'coverage_Basic', 'coverage_Extended',
       'coverage_Premium', 'education_Bachelor', 'education_College',
       'education_Doctor', 'education_High School or Below',
       'education_Master', 'employment_status_Disabled',
       'employment_status_Employed', 'employment_status_Medical Leave',
       'employment_status_Retired', 'employment_status_Unemployed', 'gender_F',
       'gender_M', 'location_code_Rural', 'location_code_Suburban',
       'location_code_Urban', 'marital_status_Divorced',
       'marital_status_Married', 'marital_status_Single',
       'policy_type_Corporate Auto', 'policy_type_Personal Auto',
       'policy_type_Special Auto', 'policy_Corporate L1',
       'policy_Corporate L2', 'policy_Corporate L3', 'policy_Personal L1',
       'policy_Personal L2', 'policy_Personal L3', 'policy_Special L1',
       'policy_Special L2', 'policy_Special L3', 'renew_offer_type_Offer1',
       'renew_offer_type_Offer2', 'renew_offer_type_O

In [110]:
# removing columns to get better result 
New_df = New_df.drop(['policy_type_Corporate Auto', 'policy_type_Personal Auto',
       'policy_type_Special Auto', 'policy_Corporate L1',
       'policy_Corporate L2', 'policy_Corporate L3', 'policy_Personal L1',
       'policy_Personal L2', 'policy_Personal L3', 'policy_Special L1',
       'policy_Special L2', 'policy_Special L3', 'renew_offer_type_Offer1',
       'renew_offer_type_Offer2', 'renew_offer_type_Offer3',
       'renew_offer_type_Offer4', 'sales_channel_Agent',
       'sales_channel_Branch', 'sales_channel_Call Center',
       'sales_channel_Web'], axis = 1)


In [111]:
New_df

,response_No,response_Yes,coverage_Basic,coverage_Extended,coverage_Premium,education_Bachelor,education_College,education_Doctor,education_High School or Below,education_Master,...,vehicle_type_A,vehicle_type_Other vehicule,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.035752,0.480381,0.000000,0.200000,0.525253,0.0,1.000,292.8
1,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.004059,0.000000,0.012658,0.085714,0.262626,0.0,0.000,744.9
2,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.160264,0.221432,0.164557,0.971429,0.313131,0.0,0.125,480.0
3,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.250952,0.490873,0.151899,0.285714,0.030303,0.0,0.125,484.0
4,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.087527,0.236795,0.236287,0.431429,0.313131,0.2,0.750,707.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.167823,0.000000,0.810127,0.431429,0.404040,0.2,0.750,1214.4
10906,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.041281,0.611576,0.016878,0.200000,0.686869,0.0,0.625,273.0
10907,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.270122,0.398446,0.590717,0.314286,0.636364,0.0,0.125,381.3
10908,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.123718,0.642072,0.409283,0.000000,0.272727,0.8,0.625,618.3


In [112]:
# Split the data into features (X) and the target variable (Y)
Y = New_df['total_claim_amount']
X = New_df.drop(['total_claim_amount'], axis=1)

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [113]:
X = sm.add_constant(X)
model = sm.OLS(Y,X).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     total_claim_amount   R-squared:                       0.770
Model:                            OLS   Adj. R-squared:                  0.769
Method:                 Least Squares   F-statistic:                     1138.
Date:                Tue, 07 Nov 2023   Prob (F-statistic):               0.00
Time:                        13:59:38   Log-Likelihood:                -69404.
No. Observations:               10910   AIC:                         1.389e+05
Df Residuals:                   10877   BIC:                         1.391e+05
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const                           -1.587e+14   1.58e+14     -1.006      0.314   -4.68e+14    1.51e+14
response_No                     -8.152e+13    8.1e+13     -1.006      0.314    -2.4e+14    7.73e+13
response_Yes                    -8.152e+13    8.1e+13     -1.006      0.314    -2.4e+14    7.73e+13
coverage_Basic                  -5.728e+13   5.69e+13     -1.006      0.314   -1.69e+14    5.43e+13
coverage_Extended               -5.728e+13   5.69e+13     -1.006      0.314   -1.69e+14    5.43e+13
coverage_Premium                -5.728e+13   5.69e+13     -1.006      0.314   -1.69e+14    5.43e+13
education_Bachelor              -1.121e+13   1.11e+13     -1.006      0.314   -3.31e+13    1.06e+13
education_College               -1.121e+13   1.11e+13     -1.006      0.314   -3.31e+13    1.06e+13
education_Doctor                -1.121e+13   1.11e+13     -1.006      0.314   -3.31e+13    1.06e+13
education_High School or Below  -1.121e+13   1.11e+13     -1.006      0.314   -3.31e+13    1.06e+13
education_Master                -1.121e+13   1.11e+13     -1.006      0.314   -3.31e+13    1.06e+13
employment_status_Disabled       2.255e+13   2.24e+13      1.006      0.314   -2.14e+13    6.65e+13
employment_status_Employed       2.255e+13   2.24e+13      1.006      0.314   -2.14e+13    6.65e+13
employment_status_Medical Leave  2.255e+13   2.24e+13      1.006      0.314   -2.14e+13    6.65e+13
employment_status_Retired        2.255e+13   2.24e+13      1.006      0.314   -2.14e+13    6.65e+13
employment_status_Unemployed     2.255e+13   2.24e+13      1.006      0.314   -2.14e+13    6.65e+13
gender_F                         6.708e+13   6.67e+13      1.006      0.314   -6.36e+13    1.98e+14
gender_M                         6.708e+13   6.67e+13      1.006      0.314   -6.36e+13    1.98e+14
location_code_Rural              4.793e+13   4.77e+13      1.006      0.314   -4.55e+13    1.41e+14
location_code_Suburban           4.793e+13   4.77e+13      1.006      0.314   -4.55e+13    1.41e+14
location_code_Urban              4.793e+13   4.77e+13      1.006      0.314   -4.55e+13    1.41e+14
marital_status_Divorced          4.342e+13   4.32e+13      1.006      0.314   -4.12e+13    1.28e+14
marital_status_Married           4.342e+13   4.32e+13      1.006      0.314   -4.12e+13    1.28e+14
marital_status_Single            4.342e+13   4.32e+13      1.006      0.314   -4.12e+13    1.28e+14
vehicle_class_Car                1.797e+13   1.79e+13      1.006      0.314    -1.7e+13     5.3e+13
vehicle_class_Four-Door Car      1.797e+13   1.79e+13      1.006      0.314    -1.7e+13     5.3e+13
vehicle_class_Luxury Car         1.797e+13   1.79e+13      1.006      0.314    -1.7e+13     5.3e+13
vehicle_class_Luxury SUV         1.797e+13   1.79e+13      1.006      0.314    -1.7

In [114]:
# Evaluate the model again result are the same , could not perform a better model
r2 = r2_score(Y_test, Y_pred)
mse = mean_squared_error(Y_test, Y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y_test, Y_pred)

# Print evaluation metrics
print("R-squared (R2) =", round(r2, 4))
print("Mean Squared Error (MSE) =", round(mse, 2))
print("Root Mean Squared Error (RMSE) =", round(rmse, 2))
print("Mean Absolute Error (MAE) =", round(mae, 2))

R-squared (R2) = -2.4086802244912767e+19
Mean Squared Error (MSE) = 1.9448493480996368e+24
Root Mean Squared Error (RMSE) = 1394578555729.16
Mean Absolute Error (MAE) = 43846493274.19
